# PowerBI

## How to Show Last Refresh Date in Power BI
Step 1: Open the Power BI desktop, Under the **Home** ribbon, click on the **Get data** for Common data sources option and select **Blank query**

Step 2: Once you have Blank Query table Query1 in place under the Queries section, right-click on it and rename it to **Last Refreshed**

Step 3: Afterward, go to the formula bar, write below the M query, and press enter.
```sql
= DateTime.LocalNow()
```

## How to Custom Date Period Selections in Power BI


Reference : 
    - https://www.youtube.com/watch?v=fKygF7VEJnQ
    - https://www.linkedin.com/pulse/custom-range-date-slicer-power-bi-dynamic-defaults-more-a%C5%9F%C4%B1ro%C4%9Flu

1. Create a dataset called **dim_date** with unique date range only 

```pyhton
dim_date = DISTINCT( CALENDAR( MIN( datamart_name [REPORT_TIME] ), MAX( datamart_name[REPORT_TIME] ) ))
```

and change the Date column data type to Date


2. Create a new table called **date_slicer**

```java
date_filter = 
VAR _today = TODAY()
VAR _thisweekstart = TODAY() - WEEKDAY(TODAY(),2) + 1
VAR _lastweekstart = TODAY() - WEEKDAY(TODAY(),2) - 6
VAR _mtd = DATESMTD( _dim_date[Date] )
VAR _qtd = DATESQTD( _dim_date[Date] )
VAR _ytd = DATESYTD( _dim_date[Date] )
VAR _all = CALENDAR( MIN(_dim_date[Date] ), MAX(_dim_date[Date]) ) 


RETURN 
    UNION(
        ADDCOLUMNS( FILTER( _dim_date, [Date] >= _thisweekstart ), "Period", "This Week", "Order", 0),
        ADDCOLUMNS( FILTER( _dim_date, AND([Date]>= _lastweekstart, [Date]< _thisweekstart)), "Period", "Last Week", "Order",1),
        ADDCOLUMNS( _mtd, "Period", "Month to Date", "Order", 2),
        ADDCOLUMNS( PREVIOUSMONTH( _mtd), "Period", "Last Month", "Order",3),
        ADDCOLUMNS( _qtd, "Period", "Quarter to Date", "Order", 4),
        ADDCOLUMNS( PREVIOUSQUARTER( _qtd), "Period", "Last Quarter", "Order", 5),
        ADDCOLUMNS( _ytd, "Period", "Year to Date", "Order", 6),
        ADDCOLUMNS( _all, "Period", "Custom Range", "Order", 7)
        )
```

# How to create group by day/week/month

create few new metrics in the datamart or fact table datasource

```JAVA
_week_number = 
    IF(
        WEEKNUM(FACT_GSC_PAGE[tx_date], 2) = 53, 
        1, 
        WEEKNUM(FACT_GSC_PAGE[tx_date],2)
    )

    
_week_year = 
    IF(
        AND(FACT_GSC_PAGE[_week_number] = 1, MONTH(FACT_GSC_PAGE[tx_date]) = 12),
        YEAR(FACT_GSC_PAGE[tx_date]) + 1,
        YEAR(FACT_GSC_PAGE[tx_date])
    )
    
    
_report_week = 
    CALCULATE (
        MIN ( FACT_GSC_PAGE[tx_date] ),
        ALLEXCEPT ( FACT_GSC_PAGE, FACT_GSC_PAGE[_week_number], FACT_GSC_PAGE[_week_year] )
    )  
    
_report_month = FORMAT(FACT_GSC_PAGE[tx_date], "yyyy-mm-01")
```

3. create a New Measure in the created dim_date table 

```python 
date_slicer_filter = 
IF( 
    SELECTEDVALUE(date_filter[Period]) = "Custom Range",
    1, 
    0 
    )
```

4. You must set up the relationships between **date_filter** & **dim_date**

    - Cardinality : Many to 1 
    - Cross Filter Direction : **Both**

## How to get the max value of a group of measure 

https://datacaffee.com/sql-equivalent-power-bi-dax-functions-part-2/#:~:text=First_value,group%20or%20within%20the%20group.

```sql 
_round_settle_status = 
        CALCULATE( MAXX(table_name, [column_name]), ALLEXCEPT( table_name , table_name[dim_col_1] , table_name[dim_col_2]))

```

```sql
validate_cash = 

VAR _change = 
        CALCULATE(  
            'ALL MEASURES'[_balance_change_cash] ,
            DIM_CHECKS, DIM_CHECKS[SECTION] = "BALANCE CHANGE",
            DIM_CHECKS, DIM_CHECKS[SECTION] <> "RECONCILIATION CHECK"
            ) 

VAR _recon = 
        CALCULATE(  
            'ALL MEASURES'[_recon_check_cash],
            DIM_CHECKS, DIM_CHECKS[SECTION] = "RECONCILIATION CHECK",
            DIM_CHECKS, DIM_CHECKS[SECTION] <> "BALANCE CHANGE"
            ) 

RETURN 
    _change 
```

```sql

_color_label = 

    VAR _opening = SELECTEDVALUE(_dim_sections[SECTION], "OPENING BALANCE") 
    VAR _if_opending = IF( _opening = "OPENING BALANCE", 1, 0)

    RETURN 
        "<b>" & _if_opending &"</b>"

```

```sql
__Total Budget EST2 Monthly = 
SWITCH(TRUE();
[Selected currency]="EUR"; FORMAT([_Total Budget EST2M]/VALUES('Currency'[Currency Rate]);"€ 0,00");
[Selected currency]="USD"; FORMAT([_Total Budget EST2M]/VALUES('Currency'[Currency Rate]);"U$ 0,00");
[Selected currency]="DKK"; FORMAT([_Total Budget EST2M]/VALUES('Currency'[Currency Rate]);"0,00 kr");
[Selected currency]="BRL"; FORMAT([_Total Budget EST2M]/VALUES('Currency'[Currency Rate]);"R$ 0,00");
[Selected currency]="CAD"; FORMAT([_Total Budget EST2M]/VALUES('Currency'[Currency Rate]);"C$ 0,00");
[Selected currency]="CHF"; FORMAT([_Total Budget EST2M]/VALUES('Currency'[Currency Rate]);"0,00 fr");
[Selected currency]="GBP"; FORMAT([_Total Budget EST2M]/VALUES('Currency'[Currency Rate]);"£ 0,00");
[Selected currency]="AUD"; FORMAT([_Total Budget EST2M]/VALUES('Currency'[Currency Rate]);"A$ 0,00");
FORMAT([_Total Budget 2019M];"R$ 0,00"));FORMAT([_Total Budget EST2M];"R$ 0,00"))
```

# How to Text formatting with DAX ( Bold)

- https://bestinbi.es/en/blog/text-formatting-with-dax/ 
- https://yaytext.com/
- https://unicodelookup.com/

```sql 
_calculated_Cash = 
    VAR _others =  
        CALCULATE(  
            [cash],
            DM_BALANCE_ACTIVITY_PER_HOUR[METRIC_LVL] = 2
        )
    VAR _opening = [Opening Cash]
    VAR _closing = [Closing Cash]
    VAR _total_value = _others + _opening + _closing

    VAR _section_name = SELECTEDVALUE( _dim_sections[SECTION])

    VAR _cash_text = FIXED( _total_value, 2)
    VAR _bold_0 = SUBSTITUTE(_cash_text, "0" ,UNICHAR(120782))
    VAR _bold_1 = SUBSTITUTE(_bold_0, "1" ,UNICHAR(120783))
    VAR _bold_2 = SUBSTITUTE(_bold_1, "2" ,UNICHAR(120784))
    VAR _bold_3 = SUBSTITUTE(_bold_2, "3" ,UNICHAR(120785))
    VAR _bold_4 = SUBSTITUTE(_bold_3, "4" ,UNICHAR(120786))
    VAR _bold_5 = SUBSTITUTE(_bold_4, "5" ,UNICHAR(120787))
    VAR _bold_6 = SUBSTITUTE(_bold_5, "6" ,UNICHAR(120788))
    VAR _bold_7 = SUBSTITUTE(_bold_6, "7" ,UNICHAR(120789))
    VAR _bold_8 = SUBSTITUTE(_bold_7, "8" ,UNICHAR(120790))
    VAR _bold_9 = SUBSTITUTE(_bold_8, "9" ,UNICHAR(120791))
    VAR _bold_comma = SUBSTITUTE(_bold_9, "," ,UNICHAR(44))
    VAR _bold_stop = SUBSTITUTE(_bold_comma, "." ,UNICHAR(46))
    

RETURN 
    SWITCH(
        TRUE(),
        _section_name = "OPENING BALANCE", _bold_9,
        _section_name = "CLOSING BALANCE", _bold_9,
        _section_name <> "OPENING BALANCE", _total_value
    )


```

# SWITCH feature
https://www.enjoysharepoint.com/power-bi-measure-switch/